# emPAI

In [ ]:
# Load Packages and define TYRPSIN Function
import os
from tkinter import filedialog

import pandas as pd

def TRYPSIN(proseq, miss_cleavage, min_size, max_size): # default miss_cleavage = 2
    peptides=[]
    goldilocksPeptides=[]
    cut_sites=[0]
    for i in range(0,len(proseq)-1):
        if proseq[i]=='K' and proseq[i+1]!='P':
            cut_sites.append(i+1)
        elif proseq[i]=='R' and proseq[i+1]!='P':
            cut_sites.append(i+1)
    
    if cut_sites[-1]!=len(proseq):
        cut_sites.append(len(proseq))

    if len(cut_sites)>2:
        if  miss_cleavage==0:
            for j in range(0,len(cut_sites)-1):
                peptides.append(proseq[cut_sites[j]:cut_sites[j+1]])

        elif miss_cleavage==1:
            for j in range(0,len(cut_sites)-2):
                peptides.append(proseq[cut_sites[j]:cut_sites[j+1]])
                peptides.append(proseq[cut_sites[j]:cut_sites[j+2]])
            
            peptides.append(proseq[cut_sites[-2]:cut_sites[-1]])

        elif miss_cleavage==2:
            for j in range(0,len(cut_sites)-3):
                peptides.append(proseq[cut_sites[j]:cut_sites[j+1]])
                peptides.append(proseq[cut_sites[j]:cut_sites[j+2]])
                peptides.append(proseq[cut_sites[j]:cut_sites[j+3]])
            
            peptides.append(proseq[cut_sites[-3]:cut_sites[-2]])
            peptides.append(proseq[cut_sites[-3]:cut_sites[-1]])
            peptides.append(proseq[cut_sites[-2]:cut_sites[-1]])
    else: #there is no trypsin site in the protein sequence
        peptides.append(proseq)
    for i in range(0, len(peptides)): # to filter for peptide size (default = 6 < x < 50)
        if len(peptides[i])>min_size and len(peptides[i])<max_size:
            goldilocksPeptides.append(peptides[i])
        else:
            pass    
    return goldilocksPeptides

In [ ]:
# PATH
path = filedialog.askdirectory()

# Load PD Protein Excel File and Uniprot DB
PD_Protein_raw = filedialog.askopenfilename()
PD_Protein = pd.read_excel(PD_Protein_raw)
Uniprot_Seq = pd.read_excel('Hu_Uniprot_Sequence_geneSymbol.xlsx')

# Merge
PDUmerged = Uniprot_Seq.merge(PD_Protein)

### TRYPSIN Function with parameters; 
#DataFrame, # of missed cleavages allowed, min peptide size, and max peptide size
Peptide_List=[]
for i in range(len(PDUmerged)):
    Peptide_List.append(TRYPSIN(PDUmerged.Sequence[i],2,6,50))

# get n Obervable Proteins for emPAI calculation    
n_observable = []
for i in range(len(Peptide_List)):
    n_observable.append(len(Peptide_List[i]))    
    
# emPAI Calculation    
emPAI = []
for i in range(len(PDUmerged)):
    emPAI.append((10**(PDUmerged['# Unique Peptides'][i]/n_observable[i]))-1)    
    
# Append new column to DataFrame; clean and write to Excel
PDUmerged['emPAI'] = emPAI
PDUmerged.pop('Unnamed: 0')
PDUmerged.to_excel(f'{path}/Protein_emPAI.xlsx')